<a href="https://colab.research.google.com/github/Hesh0629/Hesh0629/blob/main/titanic_(2nd_try).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_data=pd.read_csv('/content/drive/MyDrive/titanic_train.csv')
test_data=pd.read_csv('/content/drive/MyDrive/titanic_test.csv')
total_data=[train_data,test_data]

In [ ]:
print(train_data.isnull().sum(),test_data.isnull().sum())
print(train_data,test_data)

In [ ]:
sex_mapping = {"male": 0, "female":1}
em_mapping = {"S": 0, "C":1,"Q":2}
title_mapping = {"Mr":0, "Miss":1, "Mrs":2,
                "Master":3, "Dr":3, "Rev":3, "Col": 3, 'Ms': 3, 'Mlle': 3, "Major": 3, 'Lady': 3, 'Capt': 3,
                 'Sir': 3, 'Don': 3, 'Mme':3, 'Jonkheer': 3, 'Countess': 3 }
for dataset in total_data:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)
    dataset['Embarked'] = dataset['Embarked'].map(em_mapping)
    dataset['Title'] = dataset['Name'].str.extract('([A-za-z]+)\.', expand=False)
    dataset['Title'] = dataset['Title'].map(title_mapping)

**2021.04.17  
Cabin, Ticket을 날려먹었는데 이 부분에서 잘못된 듯 하다**

In [ ]:
train_data.drop('Name',axis=1,inplace=True)
train_data.drop('Cabin',axis=1,inplace=True)
train_data.drop('Ticket',axis=1,inplace=True)
test_data.drop('Name',axis=1,inplace=True)
test_data.drop('Cabin',axis=1,inplace=True)
test_data.drop('Ticket',axis=1,inplace=True)

In [ ]:
train_label=train_data['Survived']
train_data.drop('Survived',axis=1,inplace=True)

In [ ]:
train_data.drop('PassengerId',axis=1,inplace=True)
test_id=test_data['PassengerId']
test_data.drop('PassengerId',axis=1,inplace=True)

In [ ]:
train_data['Age'].fillna(train_data.groupby('Title')['Age'].transform('median'), inplace=True)
test_data['Age'].fillna(test_data.groupby('Title')['Age'].transform('median'), inplace=True)
train_data['Embarked']=train_data['Embarked'].fillna(0)
test_data['Embarked']=test_data['Embarked'].fillna(0)
test_data['Fare'].fillna(test_data.mean()['Fare'], inplace=True)
test_data['Title'].fillna(3,inplace=True)

In [ ]:
train_data=(train_data-train_data.mean())/train_data.std()
test_data=(test_data-train.mean())/train.std()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

tree=RandomForestClassifier()
params = { 'n_estimators' : [10,11,12,13,14,15,20,25],
           'max_depth' : [7,8,9,10,11,12,13,14,15,16,17,18,19,20], #깊을수록 과적합
           'min_samples_leaf' : [2,3,4,5,6,7,8], #과적합 제어용, 작을수록 과적합
           'min_samples_split' :  [1,2,3,4,5,6,7,8] #적을수록 과적합
            } #파라미터 범위를 설정
          
grid_cv=GridSearchCV(tree, param_grid=params,cv=4,n_jobs=-1) #CV는 Kfold에서 K와 같음, 
grid_cv.fit(train_data,train_label)
 
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))
tree=grid_cv.best_estimator_

In [ ]:
tree=grid_cv.best_estimator_
result=tree.predict(test_data)

In [ ]:
res=pd.DataFrame(data={'PassengerId':test_id.astype('int'), 'Survived':result})
res.to_csv('Prediction_4.csv',index=False)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

boost=XGBClassifier()
param={'booster' :['gbtree'],  #부스팅 과정에서 이용할 트리종류 (수정x)
                 'silent':[True], #verbose랑 같다
                 'max_depth':[4,5,6,8,9], #
                 'min_child_weight':[1,3,5,7,9,11], #값이 높을수록 과소적합 발생
                 'gamma':[0.5,1,1.5,2,2.5], #노드가 split 되기 위한 loss function의 값이 감소하는 최소값. 클수록 모델이 덜 바뀜
                 'nthread':[1], #XG Boost를 실행하기 위한 cpu 스레드 개수
                 #'eta':[0.0001,0.0005],
                 #'colsample_bytree':[0.9], # 트리 생성시 훈련 데이터에서 변수를 샘플링 하는 비율 (0~1)
                 #'colsample_bylevel':[0.9], # 트리의 레벨별로 훈련 데이터의 변수를 샘플링 하는 비율
                 'n_estimators':[450,500,550,600], #트리 개수
                 'objective':['binary:logistic'], #회귀인 경우 'reg:linear',이중 분류 'binary:logistic',분류된 클래스를 반환하는 경우 'multi:softmax', 클래스에 속한 확률'multi:softprob'
                 'random_state':[2]} #random seed nuber임. 숫자 고정해 두면 동일한 데이터 셋을 얻을 수 잇음
 
gcv=GridSearchCV(boost, param_grid=param, cv=3, scoring='f1', n_jobs=-1) #이진분류에서 scoring은 f1, n_jobs -1로 두면 모든 코어 조지기
gcv.fit(train_data,train_label)
print(gcv.best_params_," ",gcv.best_score_)

{'booster': 'gbtree', 'gamma': 0.5, 'max_depth': 8, 'min_child_weight': 7, 'n_estimators': 450, 'nthread': 1, 'objective': 'binary:logistic', 'random_state': 2, 'silent': True}   0.7779408567079799


In [ ]:
boost=gcv.best_estimator_
result=boost.predict(test_data)

In [ ]:
res=pd.DataFrame(data={'PassengerId':test_id.astype('int'), 'Survived':result})
res.to_csv('Prediction_4.csv',index=False)

In [304]:
from sklearn.ensemble import AdaBoostClassifier

ad=AdaBoostClassifier()
params = { 'n_estimators' : [700,800,900,100],
          'learning_rate' : [0.8,0.7,0.6,0.5], #AdaBoost는 줄어드는 쪽으로 해줘야함
          'random_state':[2]} #파라미터 범위를 설정
          
grid_cv=GridSearchCV(ad,param_grid=params,cv=4,n_jobs=-1) #CV는 Kfold에서 K와 같음, 
grid_cv.fit(train_data,train_label)
 
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))
ad=grid_cv.best_estimator_

최적 하이퍼 파라미터:  {'learning_rate': 0.6, 'n_estimators': 800, 'random_state': 2}
최고 예측 정확도: 0.8092


https://www.kaggle.com/treina/titanic-with-adaboost  
이게뭐노

In [305]:
#ad=AdaBoostClassifier(RandomForestClassifier(n_estimators = 1000),algorithm="SAMME",n_estimators=500,learning_rate=0.5)
result=ad.predict(test_data)

In [306]:
res=pd.DataFrame(data={'PassengerId':test_id.astype('int'), 'Survived':result})
res.to_csv('Prediction_9.csv',index=False)

In [ ]:
print(train_data,test_data)